# Stock Value Prediction

In this Notebook, we will create the actual prediction system, by testing various approaches and accuracy against multiple time-horizons (target_days variable).

First we will load all libraries:

In [3]:
import pandas as pd
import numpy as np
import sys, os
from datetime import datetime
sys.path.insert(1, '..')
import recommender as rcmd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# classification approaches
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

# regression approaches
from sklearn.linear_model import LinearRegression

# data handling and scoring
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, precision_score, f1_score, mean_squared_error


Next, we create the input data pipelines for stock and statement data. Therefore we will have to split data into training and test sets. There are two options for doing that:

* Splitting the list of symbols
* Splitting the results list of training stock datapoints

We will use the first option in order ensure a clear split (since the generate data has overlapping time frames, the second options would generate data that might have been seen by the system beforehand).

In [4]:
# create cache object
cache = rcmd.stocks.Cache()

# load list of all available stocks and sample sub-list
stocks = cache.list_data('stock')

def train_test_data(back, ahead, xlim, split=0.3, count=2000, stocks=stocks, cache=cache):
    '''Generetes a train test split'''
    sample = np.random.choice(list(stocks.keys()), 2000)
    # split the stock data
    count_train = int((1-split) * count)
    sample_train = sample[:count_train]
    sample_test = sample[count_train:]

    # generate sample data
    df_train = rcmd.learning.preprocess.create_dataset(sample_train, stocks, cache, back, ahead, xlim)
    df_test = rcmd.learning.preprocess.create_dataset(sample_test, stocks, cache, back, ahead, xlim)
    
    return df_train, df_test

df_train, df_test = train_test_data(14, 66, (-.5, .5))
print(df_train.shape)
df_train.head()

[-0.5  -0.25  0.    0.25  0.5 ]


..\recommender\learning\preprocess.py:252: RuntimeWarning: divide by zero encountered in true_divide
  df['expenses_research_netcash'] = np.divide(df['expenses_research'], df['cash_net'])
..\recommender\learning\preprocess.py:252: RuntimeWarning: invalid value encountered in true_divide
  df['expenses_research_netcash'] = np.divide(df['expenses_research'], df['cash_net'])
..\recommender\learning\preprocess.py:311: RuntimeWarning: divide by zero encountered in true_divide
  df['pe_ratio'] = np.divide(df[col_price], df['eps_diluted'])
..\recommender\learning\preprocess.py:312: RuntimeWarning: divide by zero encountered in true_divide
  df['cash_share'] = np.divide(df['cash_net'], np.divide(df['shareholder_equity'], df[col_price]))


[-0.5  -0.25  0.    0.25  0.5 ]


..\recommender\learning\preprocess.py:252: RuntimeWarning: divide by zero encountered in true_divide
  df['expenses_research_netcash'] = np.divide(df['expenses_research'], df['cash_net'])
..\recommender\learning\preprocess.py:311: RuntimeWarning: divide by zero encountered in true_divide
  df['pe_ratio'] = np.divide(df[col_price], df['eps_diluted'])
..\recommender\learning\preprocess.py:312: RuntimeWarning: divide by zero encountered in true_divide
  df['cash_share'] = np.divide(df['cash_net'], np.divide(df['shareholder_equity'], df[col_price]))


(42542, 35)


,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,...,dividend_share_growth_10y,revenue_share_growth_3y,revenue_share_growth_5y,revenue_share_growth_10y,pe_ratio,cash_share,norm_price,target,target_cat,symbol
date,,,,,,,,,,,,,,,,,,,,,
2009-05-29,-0.074448,-0.032145,-0.034584,-0.060832,-0.066147,-0.055590,-0.081219,-0.063635,-0.052241,-0.030580,...,0.0000,0.0632,0.0637,0.1173,40.395588,37.358140,27.4690,0.123134,3,ACN
2009-05-29,-0.074182,0.005387,0.051057,0.042934,0.070700,0.062329,0.043763,0.046747,0.077414,0.076668,...,0.0450,-0.0329,0.0296,-0.0178,44.685183,0.843000,12.0650,0.144475,3,APOG
2009-05-29,-0.148598,-0.148598,-0.056340,-0.056340,-0.104358,-0.104358,-0.096529,-0.096529,-0.048127,-0.048127,...,-1.0000,-0.0995,-0.0553,-0.0162,-2.099310,-0.256086,1.8264,-0.012180,2,BSET
2009-05-29,-0.002762,0.058128,0.093246,0.084071,0.105302,0.038836,0.019425,0.011060,0.023155,0.044372,...,-0.0024,-0.0427,0.0081,0.0634,28.968078,0.759693,7.5317,0.423535,4,EBF
2009-05-29,0.028571,0.028571,-0.057143,-0.057143,-0.114286,-0.114286,-0.028571,-0.028571,0.000000,0.000000,...,0.0000,-0.1190,0.0137,-0.0085,1.093750,9.149215,1.4000,-0.202597,2,EMMS


Now that we have loaded and split the data, we have to divide it into input and output data:

In [33]:
def divide_data(df, xlim):
    '''Splits the data into 3 sets: input, ouput_classify, output_regression.
    
    Args:
        df (DataFrame): DF to contain all relevant data
        xlim (tuple): tuple of integers used to clip and scale regression values to a range of 0 to 1
        
    Returns:
        df_X: DataFrame with input values
        df_y_cls: DataFrame with classification labels
        df_y_reg: DataFrame with regression values
    '''
    # remove all target cols
    df_X = df.drop(['target', 'target_cat', 'norm_price', 'symbol'], axis=1)
    # convert to dummy classes
    df_y_cls = pd.get_dummies(df['target_cat'], prefix='cat', dummy_na=False)
    # clip values and scale to vals
    df_y_reg = np.divide( np.subtract( df['target'].clip(xlim[0], xlim[1]), xlim[0] ), (xlim[1] - xlim[0]) )
    
    return df_X, df_y_cls, df_y_reg

X_train, y_ctrain, y_rtrain = divide_data(df_train, (-.5, .5))
X_test, y_ctest, y_rtest = divide_data(df_test, (-.5, .5))
y_ctrain.sum(axis=0)

cat_0      243
cat_1     2047
cat_2    15655
cat_3    20405
cat_4     3126
cat_5     1066
dtype: int64

Before we create the actual prediction systems, we will have to define metrics, how we want to measure the success of the systems.
As we have two approaches (classification and regression) we will use two types metrics:

* Precision, Recall & Accuracy
* RMSE

In [34]:
def _metric_classifier(y_true, y_pred, avg=None):
    p = precision_score(y_true, y_pred, average=avg)
    r = recall_score(y_true, y_pred, average=avg)
    f1 = f1_score(y_true, y_pred, average=avg)
    return f1, p, r

def score_classifier(y_true, y_pred):
    '''Calculates the relevant scores for a classifer and outputs. This should show predicitions per class.'''
    f1, p, r = _metric_classifier(y_true, y_pred, avg='micro')
    
    print("Model Performance: F1={:.4f} (P={:.4f} / R={:.4f})".format(f1, p, r))
    # list scores of single classes
    for i, c in enumerate(y_true.columns):
        sf1, sp, sr = _metric_classifier(y_true.iloc[:, i], y_pred[:, i], avg='binary')
        print("  {:10} F1={:.4f} (P={:.4f} / R={:.4f})".format(c + ":", sf1, sp, sr))
        
def score_regression(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print("Model Performance: MSE={:.4f}".format(mse))

## Classification

The first step is to create a baseline for both approaches (classification and regression). In case of regression our target value will be `target` and for classification it will be `target_cat` (which we might convert into a one-hot vector along the way).

Lets start with the simpler form of classification:

In [7]:
y_ctrain.sum(axis=0)

cat_0      243
cat_1     2047
cat_2    15655
cat_3    20405
cat_4     3126
cat_5     1066
dtype: int64

In [8]:
# scale input data to improve convergance (Note: scaler has to be used for other input data as well)
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# train element
classifier = MultiOutputClassifier(LogisticRegression(max_iter=500, solver='lbfgs'))
classifier.fit(X_train_std, y_ctrain)

# predict data
y_pred = classifier.predict(X_test_std)

score_classifier(y_ctest, y_pred)

Model Performance: F1=0.2655 (P=0.4987 / R=0.1809)
  cat_0:     F1=0.0099 (P=0.0122 / R=0.0083)
  cat_1:     F1=0.0138 (P=0.0654 / R=0.0077)
  cat_2:     F1=0.0116 (P=0.5735 / R=0.0059)
  cat_3:     F1=0.4302 (P=0.5175 / R=0.3682)
  cat_4:     F1=0.0026 (P=0.0455 / R=0.0014)
  cat_5:     F1=0.0000 (P=0.0000 / R=0.0000)


We can see a strong bias in the system for `cat_3`, which also has the highest number of training samples. Future work might include oversampling or more target datapoint selection to reduce these biases.

Next, support vector machines:

In [20]:
classifier_svm = MultiOutputClassifier(SVC())
classifier_svm.fit(X_train_std, y_ctrain)
y_pred_svm = classifier_svm.predict(X_test_std)
score_classifier(y_ctest, y_pred_svm)

Model Performance: F1=0.4157 (P=0.5416 / R=0.3372)
  cat_0:     F1=0.0385 (P=1.0000 / R=0.0196)
  cat_1:     F1=0.0154 (P=1.0000 / R=0.0078)
  cat_2:     F1=0.0107 (P=0.4932 / R=0.0054)
  cat_3:     F1=0.6126 (P=0.5414 / R=0.7053)
  cat_4:     F1=0.0027 (P=1.0000 / R=0.0014)
  cat_5:     F1=0.0000 (P=0.0000 / R=0.0000)


c:\programming\envs\ds-stocks\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\ds-stocks\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


We can see the results improve

In [20]:
# simple feed forward network
print(X_train.shape)
classifier_ffn = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train_std.shape[1],)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(y_ctrain.shape[1], activation=tf.nn.softmax)
])
classifier_ffn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier_ffn.fit(X_train_std, df_train['target_cat'].to_numpy(), epochs=100)

y_pred_ffn = classifier_ffn.predict(X_test_std)
y_pred_ffn = pd.get_dummies(y_pred_ffn.argmax(axis=1))
print(y_pred_ffn.sum(axis=0))
score_classifier(y_ctest, y_pred_ffn.to_numpy())

(42542, 31)
Train on 42542 samples
Epoch 1/100
42542/42542 [==============================] - 4s 84us/sample - loss: 1.1531 - accuracy: 0.4854
Epoch 2/100
42542/42542 [==============================] - 3s 70us/sample - loss: 1.1172 - accuracy: 0.4941
Epoch 3/100
42542/42542 [==============================] - 3s 81us/sample - loss: 1.1088 - accuracy: 0.4949
Epoch 4/100
42542/42542 [==============================] - 4s 90us/sample - loss: 1.1019 - accuracy: 0.4997
Epoch 5/100
42542/42542 [==============================] - 5s 111us/sample - loss: 1.0935 - accuracy: 0.5012
Epoch 6/100
42542/42542 [==============================] - 5s 114us/sample - loss: 1.0868 - accuracy: 0.5025
Epoch 7/100
42542/42542 [==============================] - 4s 84us/sample - loss: 1.0799 - accuracy: 0.5063
Epoch 8/100
42542/42542 [==============================] - 4s 90us/sample - loss: 1.0711 - accuracy: 0.5069
Epoch 9/100
42542/42542 [==============================] - 4s 84us/sample - loss: 1.0607 - accuracy

It is noteworthy that the output of the model in the test data resembles the input distribution. Lets try to improve generalization with a more complex model

In [31]:
act = tf.keras.layers.PReLU
classifier_ffn = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train_std.shape[1],)),
    tf.keras.layers.Dense(32), act(),
    tf.keras.layers.Dense(64), act(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128), act(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256), act(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128), act(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64), act(),
    tf.keras.layers.Dense(y_ctrain.shape[1], activation=tf.nn.softmax)
])
classifier_ffn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier_ffn.fit(X_train.to_numpy(), df_train['target_cat'].to_numpy(), epochs=200)

y_pred_ffn = classifier_ffn.predict(X_test.to_numpy())
print(y_pred_ffn)
y_pred_ffn = pd.get_dummies(y_pred_ffn.argmax(axis=1))
print(y_pred_ffn.sum(axis=0))
score_classifier(y_ctest, y_pred_ffn.to_numpy())

Train on 42542 samples
Epoch 1/200
42542/42542 [==============================] - 10s 247us/sample - loss: 1.1949 - accuracy: 0.4635
Epoch 2/200
42542/42542 [==============================] - 9s 213us/sample - loss: 1.1519 - accuracy: 0.4784
Epoch 3/200
42542/42542 [==============================] - 9s 221us/sample - loss: 1.1448 - accuracy: 0.4842
Epoch 4/200
42542/42542 [==============================] - 11s 260us/sample - loss: 1.1401 - accuracy: 0.4881
Epoch 5/200
42542/42542 [==============================] - 9s 222us/sample - loss: 1.1386 - accuracy: 0.4886
Epoch 6/200
42542/42542 [==============================] - 9s 218us/sample - loss: 1.1358 - accuracy: 0.4889
Epoch 7/200
42542/42542 [==============================] - 10s 236us/sample - loss: 1.1335 - accuracy: 0.4916
Epoch 8/200
42542/42542 [==============================] - 10s 241us/sample - loss: 1.1304 - accuracy: 0.4888
Epoch 9/200
42542/42542 [==============================] - 9s 213us/sample - loss: 1.1283 - accuracy:

In [32]:
# save the model
classifier_ffn.save('../data/keras-model.h5')

## Regression

The other possible option is regression. We will test a linear regression against neural networks based on RMSE score to see how the predictions hold.

In [39]:
reg = LinearRegression()
reg.fit(X_train.iloc[:, :7].to_numpy(), y_rtrain)
y_pred_reg = reg.predict(X_test.iloc[:, :7].to_numpy())
score_regression(y_rtest, y_pred_reg)

Model Performance: MSE=0.0329


Now the neural Network

In [50]:
classifier_reg = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_train_std.shape[1],)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])
opt = tf.keras.optimizers.SGD(learning_rate=0.00000001, nesterov=False)
classifier_reg.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])
classifier_reg.fit(X_train.to_numpy(), y_rtrain.to_numpy(), epochs=20)

y_pred_reg = classifier_reg.predict(X_test.to_numpy())
score_regression(y_rtest, y_pred_reg)

Train on 42542 samples
Epoch 1/20
42542/42542 [==============================] - 8s 181us/sample - loss: 72308.8231 - accuracy: 0.0097
Epoch 2/20
42542/42542 [==============================] - 6s 146us/sample - loss: 3158.4172 - accuracy: 0.0091
Epoch 3/20
42542/42542 [==============================] - 7s 155us/sample - loss: 7.5809 - accuracy: 0.0061
Epoch 4/20
42542/42542 [==============================] - 7s 156us/sample - loss: 5.0027 - accuracy: 0.0062
Epoch 5/20
42542/42542 [==============================] - 6s 151us/sample - loss: 4.0884 - accuracy: 0.0061
Epoch 6/20
42542/42542 [==============================] - 7s 161us/sample - loss: 3.5077 - accuracy: 0.0063
Epoch 7/20
42542/42542 [==============================] - 6s 140us/sample - loss: 2.9895 - accuracy: 0.0062
Epoch 8/20
42542/42542 [==============================] - 7s 165us/sample - loss: 2.7040 - accuracy: 0.0061
Epoch 9/20
42542/42542 [==============================] - 7s 169us/sample - loss: 2.4751 - accuracy: 0.006

Model Performance: MSE=3.2631


In [51]:
y_pred_reg

array([[ 0.10986691],
       [ 0.23991095],
       [-0.34694692],
       ...,
       [ 0.0956597 ],
       [-0.04288995],
       [ 0.15074253]], dtype=float32)

In [52]:
y_pred_reg.shape

(18622, 1)

In [53]:
y_pred_ffn = classifier_ffn.predict(X_test.to_numpy())
print(y_pred_ffn)

[[0.00190905 0.0575433  0.4032011  0.48665118 0.04683916 0.00385619]
 [0.00347802 0.07319234 0.39260495 0.47064242 0.05408182 0.00600046]
 [0.00484556 0.07138    0.38683167 0.45450288 0.06774367 0.01469623]
 ...
 [0.00111771 0.03497788 0.40016484 0.5337459  0.0289243  0.00106938]
 [0.00483004 0.07679388 0.38836578 0.45680222 0.06269442 0.01051365]
 [0.00092138 0.0323066  0.41052336 0.5202334  0.03445359 0.00156169]]
